# GRADED ASSIGNMENT - 2

IMPLEMENT GRAPH ALGORITHM :

Consider the following graph:

![img](./Graph.jpeg)


1. Represent the Graph using Adjacency Matrix and List.
2. Find the shortest path using Dijkstra Algorithm:
    1.  from Node E to Node C. 
    2.  from Node D to Node A.
3. Find a Spanning Tree for the Graph and find the total weight.


Here’s the pseudocode for Dijsktra’s Algorithm:

1. Create a list of “distances” equal to the number of nodes and initialize each value to infinity
2. Set the “distance” to the starting node equal to 0
3. Create a list of “visited” nodes set to false for each node (since we haven’t visited any yet)
4. Loop through all the nodes
    1. Loop through all the nodes again, and pick the one that is the shortest distance away and not yet visited
    2. Set that node to visited
    3. Set the distance in the distance list to the distance to that node
5. The original “distance” list should now contain the shortest distance to each node or infinity if a node is unreachable from the desired starting node

### Adjacency List to Adjacency Matrix

In [3]:
from collections import defaultdict

In [4]:
class Node:
    def __init__(self, data, indexLoc = None):
        self.data = data
        self.indexLoc = indexLoc

In [10]:
class Graph:

    @classmethod
    def create_from_nodes(self, nodes):
        return Graph(len(nodes), len(nodes), nodes)

  
    def __init__(self, row, col, nodes = None):
        # set up an adjacency matrix
        self.adj_mat = [[0] * col for _ in range(row)]
        self.nodes = nodes
        for i in range(len(self.nodes)):
            self.nodes[i].index = i

    # Conncects from node1 to node2
    # Note row is source, column is destination
    # Updated to allow weighted edges (supporting dijkstra's alg)
    def connect_dir(self, node1, node2, weight = 1):
        node1, node2 = self.get_index_from_node(node1), self.get_index_from_node(node2)
        self.adj_mat[node1][node2] = weight
  
    # Optional weight argument to support dijkstra's alg
    def connect(self, node1, node2, weight = 1):
        self.connect_dir(node1, node2, weight)
        self.connect_dir(node2, node1, weight)

    # Get node row, map non-zero items to their node in the self.nodes array
    # Select any non-zero elements, leaving you with an array of nodes
    # which are connections_to (for a directed graph)
    # Return value: array of tuples (node, weight)
    def connections_from(self, node):
        node = self.get_index_from_node(node)
        return [(self.nodes[col_num], self.adj_mat[node][col_num]) for col_num in range(len(self.adj_mat[node])) if self.adj_mat[node][col_num] != 0]

    # Map matrix to column of node
    # Map any non-zero elements to the node at that row index
    # Select only non-zero elements
    # Note for a non-directed graph, you can use connections_to OR
    # connections_from
    # Return value: array of tuples (node, weight)
    def connections_to(self, node):
      node = self.get_index_from_node(node)
      column = [row[node] for row in self.adj_mat]
      return [(self.nodes[row_num], column[row_num]) for row_num in range(len(column)) if column[row_num] != 0]
     
  
    def print_adj_mat(self):
      for row in self.adj_mat:
          print(row)
  
    def node(self, index):
      return self.nodes[index]
    
  
    def remove_conn(self, node1, node2):
      self.remove_conn_dir(node1, node2)
      self.remove_conn_dir(node2, node1)
   
    # Remove connection in a directed manner (nod1 to node2)
    # Can accept index number OR node object
    def remove_conn_dir(self, node1, node2):
      node1, node2 = self.get_index_from_node(node1), self.get_index_from_node(node2)
      self.adj_mat[node1][node2] = 0   
  
    # Can go from node 1 to node 2?
    def can_traverse_dir(self, node1, node2):
      node1, node2 = self.get_index_from_node(node1), self.get_index_from_node(node2)
      return self.adj_mat[node1][node2] != 0  
  
    def has_conn(self, node1, node2):
      return self.can_traverse_dir(node1, node2) or self.can_traverse_dir(node2, node1)
  
    def add_node(self,node):
      self.nodes.append(node)
      node.index = len(self.nodes) - 1
      for row in self.adj_mat:
        row.append(0)     
      self.adj_mat.append([0] * (len(self.adj_mat) + 1))

    # Get the weight associated with travelling from n1
    # to n2. Can accept index numbers OR node objects
    def get_weight(self, n1, n2):
        node1, node2 = self.get_index_from_node(n1), self.get_index_from_node(n2)
        return self.adj_mat[node1][node2]
  
    # Allows either node OR node indices to be passed into 
    def get_index_from_node(self, node):
        if not isinstance(node, Node) and not isinstance(node, int):
            raise ValueError("node must be an integer or a Node object")
        if isinstance(node, int):
            return node
        else:
            return node.index

In [12]:
a = Node("A")
b = Node("B")
c = Node("C")
d = Node("D")
e = Node("E")

graph = Graph.create_from_nodes([a,b,c,d,e])

graph.connect(a,b,3)
graph.connect(a,c,6)
graph.connect(a,d,6)
graph.connect(a,e,1)
graph.connect(b,c,2)
graph.connect(b,d,5)
graph.connect(b,e,4)
graph.connect(c,d,6)
graph.connect(c,e,5)
graph.connect(d,e,4)

graph.print_adj_mat()

[0, 3, 6, 6, 1]
[3, 0, 2, 5, 4]
[6, 2, 0, 6, 5]
[6, 5, 6, 0, 4]
[1, 4, 5, 4, 0]


## Dijkstra Algorithm

In [29]:
class Graph:
 
    # A utility function to find the
    # vertex with minimum dist value, from
    # the set of vertices still in queue
    def minDistance(self,dist,queue):
        # Initialize min value and min_index as -1
        minimum = float("Inf")
        min_index = -1
         
        # from the dist array,pick one which
        # has min value and is till in queue
        for i in range(len(dist)):
            if dist[i] < minimum and i in queue:
                minimum = dist[i]
                min_index = i
        return min_index
 
 
    # Function to print shortest path
    # from source to j
    # using parent array
    def printPath(self, parent, j):
         
        #Base Case : If j is source
        if parent[j] == -1 :
            print(j,end=" ")
            return
        self.printPath(parent , parent[j])
        print (j,end=" ")
         
 
    # A utility function to print
    # the constructed distance
    # array
    def printSolution(self, dist, parent, src):
        src = src
        print("Vertex \t\tDistance from Source\tPath")
        for i in range(0, len(dist)):
            print("\n%d --> %d \t\t%d \t\t" % (src, i, dist[i]),end=" ")
            self.printPath(parent,i)
 
 
    '''Function that implements Dijkstra's single source shortest path
    algorithm for a graph represented using adjacency matrix
    representation'''
    def dijkstra(self, graph, src):
 
        row = len(graph)
        col = len(graph[0])
 
        # The output array. dist[i] will hold
        # the shortest distance from src to i
        # Initialize all distances as INFINITE
        dist = [float("Inf")] * row
 
        #Parent array to store
        # shortest path tree
        parent = [-1] * row
 
        # Distance of source vertex
        # from itself is always 0
        dist[src] = 0
     
        # Add all vertices in queue
        queue = []
        for i in range(row):
            queue.append(i)
             
        #Find shortest path for all vertices
        while queue:
 
            # Pick the minimum dist vertex
            # from the set of vertices
            # still in queue
            u = self.minDistance(dist,queue)
 
            # remove min element    
            queue.remove(u)
 
            # Update dist value and parent
            # index of the adjacent vertices of
            # the picked vertex. Consider only
            # those vertices which are still in
            # queue
            for i in range(col):
                '''Update dist[i] only if it is in queue, there is
                an edge from u to i, and total weight of path from
                src to i through u is smaller than current value of
                dist[i]'''
                if graph[u][i] and i in queue:
                    if dist[u] + graph[u][i] < dist[i]:
                        dist[i] = dist[u] + graph[u][i]
                        parent[i] = u
 
 
        # print the constructed distance array
        self.printSolution(dist,parent, src)

In [31]:
g= Graph()
 
graph = [[0, 3, 6, 6, 1],
         [3, 0, 2, 5, 4],
         [6, 2, 0, 6, 5],
         [6, 5, 6, 0, 4],
         [1, 4, 5, 4, 0]]

### From Node E

In [32]:
# Print the solution
g.dijkstra(graph,4)

Vertex 		Distance from Source	Path

4 --> 0 		1 		 4 0 
4 --> 1 		4 		 4 1 
4 --> 2 		5 		 4 2 
4 --> 3 		4 		 4 3 
4 --> 4 		0 		 4 

We can see that:

|4 --> 2 	|	5 	|	 4 2|

So, from E to C, we have to followthe direct path with minimum distance = 5

### From Node D

In [33]:
# Print the solution
g.dijkstra(graph,3)

Vertex 		Distance from Source	Path

3 --> 0 		5 		 3 4 0 
3 --> 1 		5 		 3 1 
3 --> 2 		6 		 3 2 
3 --> 3 		0 		 3 
3 --> 4 		4 		 3 4 

We can see that:

|3 --> 0 	|	5 	|	 3 4 0|

So, from D to A, we have to go through 4 with a minimum distance of 5